In [1]:
# Importing the required libraries
import requests
import json
import pandas as pd
import time
import datetime
import os
import numpy as np

# Universalis API no key required
# Setting the API Key for XIVAPI from the environment variable XIVAPI_KEY
xivapi_key = os.environ.get('XIVAPI_KEY')

# Setting the URL for the Universalis API
# https://docs.universalis.app/
univeralis_url = 'https://universalis.app/api/'

# Setting the URL for the XIVAPI
# https://xivapi.com/docs
xivapi_url = 'https://xivapi.com/'

In [2]:
tinctures = pd.read_csv('..\\data\\tinctures.csv')
tinctures

,ID,Icon,Name,Url,UrlType,_,_Score
0,39727,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0


In [3]:
# now to get the data for all the marketable consumables
# Initialize an empty df to store all results
tinc_results = pd.DataFrame()

# Initialize the Data center to Aether
# Initialize the histInSeconds to 1 years

world = "Aether"
years = 4
histInSeconds = years*365*24*60*60 # year in seconds

# Loop through all the marketable consumables
# Each request will be for a single consumable
# We have 2321 marketable consumables and we have to limit the requests to 25 per second
# We will use a sleep timer to limit the requests to 25 per second
failed_requests = []

for index, row in tinctures.iterrows():
    # Make the request
    itemID = row['ID']
    response = requests.get(f'https://universalis.app/api/v2/history/{world}/{itemID}?entriesToReturn=999999&entriesWithin={histInSeconds}&minSalePrice=0&maxSalePrice=2147483647')
    # check if the request was successful
    if response.status_code != 200:
        print(f"Request failed for {itemID}")
        # store the itemID in a list to retry later
        failed_requests.append(itemID)
        continue
    response = response.json()
    # normalize the data
    response = pd.json_normalize(response)
    # concatenate the data to the all_results df
    tinc_results = pd.concat([tinc_results, response,], ignore_index=True)
    time.sleep(0.04)


In [4]:
tinc_results

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.43,stackSizeHistogramNQ.52,stackSizeHistogramNQ.53,stackSizeHistogramNQ.56,stackSizeHistogramNQ.73,stackSizeHistogramNQ.22,stackSizeHistogramNQ.44,stackSizeHistogramNQ.47,stackSizeHistogramNQ.62,stackSizeHistogramNQ.83
0,39727,1710954737843,"[{'hq': True, 'pricePerUnit': 2605, 'quantity'...",Aether,32221.285000,78.142860,32143.143000,201,135,1717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39728,1710955371396,"[{'hq': True, 'pricePerUnit': 2863, 'quantity'...",Aether,12956.286000,20.571428,12935.714000,101,94,919,...,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
2,39729,1710951391617,"[{'hq': True, 'pricePerUnit': 2974, 'quantity'...",Aether,24.714285,0.000000,24.714285,16,7,74,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39730,1710954255628,"[{'hq': True, 'pricePerUnit': 2823, 'quantity'...",Aether,10521.143000,48.714287,10472.429000,62,92,784,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
4,39731,1710954153097,"[{'hq': True, 'pricePerUnit': 3089, 'quantity'...",Aether,13610.714000,21.571428,13589.143000,81,103,934,...,1.0,NaN,2.0,NaN,NaN,2.0,1.0,3.0,1.0,2.0


In [5]:
# using the tinctures df to add the name of the item to the results
tinc_results = tinc_results.merge(tinctures, how='left', left_on='itemID', right_on='ID')
tinc_results = tinc_results.drop(columns=['ID'])
tinc_results

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.44,stackSizeHistogramNQ.47,stackSizeHistogramNQ.62,stackSizeHistogramNQ.83,Icon,Name,Url,UrlType,_,_Score
0,39727,1710954737843,"[{'hq': True, 'pricePerUnit': 2605, 'quantity'...",Aether,32221.285000,78.142860,32143.143000,201,135,1717,...,NaN,NaN,NaN,NaN,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,1710955371396,"[{'hq': True, 'pricePerUnit': 2863, 'quantity'...",Aether,12956.286000,20.571428,12935.714000,101,94,919,...,NaN,NaN,NaN,NaN,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,1710951391617,"[{'hq': True, 'pricePerUnit': 2974, 'quantity'...",Aether,24.714285,0.000000,24.714285,16,7,74,...,NaN,NaN,NaN,NaN,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,1710954255628,"[{'hq': True, 'pricePerUnit': 2823, 'quantity'...",Aether,10521.143000,48.714287,10472.429000,62,92,784,...,NaN,NaN,NaN,NaN,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,1710954153097,"[{'hq': True, 'pricePerUnit': 3089, 'quantity'...",Aether,13610.714000,21.571428,13589.143000,81,103,934,...,1.0,3.0,1.0,2.0,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0


In [6]:
strength_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Strength")].values[0])
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
strength_history.to_csv('..\\data\\strength_tincture_market.csv', index=False)
strength_history


,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2605,30,Nano Clyo,False,2024-03-20 16:43:17,Adamantoise,73
1,True,2784,99,Popowazu Lalawazu,False,2024-03-20 16:34:39,Siren,57
2,True,2784,99,X'nohsoeh Bryn,False,2024-03-20 15:51:56,Siren,57
3,True,2715,25,Ryu Okami,False,2024-03-20 13:56:40,Gilgamesh,63
4,True,2799,99,Beatrix D'oraguille,False,2024-03-20 13:14:09,Cactuar,79
...,...,...,...,...,...,...,...,...
173274,True,25000,10,Gwyn Graves,False,2023-05-23 13:53:16,Gilgamesh,63
173275,True,25000,50,Gwyn Graves,False,2023-05-23 13:53:15,Gilgamesh,63
173276,False,6969,3,Xeno Izanagi,False,2023-05-23 12:39:47,Gilgamesh,63
173277,True,19997,15,Sonya Windwalker,False,2023-05-23 11:50:44,Gilgamesh,63


In [7]:
vitality_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Vitality")].values[0])
#convert the timestamp to a date
vitality_history['timestamp'] = pd.to_datetime(vitality_history['timestamp'], unit='s')
# save the data to a csv
vitality_history.to_csv('..\\data\\Vitality_tincture_market.csv', index=False)
vitality_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2974,20,Reina Uberall,False,2024-03-17 23:06:14,Gilgamesh,63
1,True,1850,30,Elsha Linnette,False,2024-03-14 12:58:43,Midgardsormr,65
2,True,2996,24,Samara Hana,False,2024-03-14 12:17:15,Adamantoise,73
3,True,2997,99,Samara Hana,False,2024-03-14 12:17:14,Adamantoise,73
4,True,2000,33,Sair Hyskaris,False,2024-03-13 16:39:09,Gilgamesh,63
...,...,...,...,...,...,...,...,...
790,True,25000,3,Jatoro Brathgar,False,2023-05-24 21:35:07,Jenova,40
791,True,6000,3,Kyro Tachikake,False,2023-05-24 03:08:11,Jenova,40
792,True,5105,3,Ilyesen Valenroix,False,2023-05-24 00:33:49,Cactuar,79
793,True,30000,3,Ronin Lothbrook,False,2023-05-23 20:35:10,Siren,57


In [8]:
dexterity_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Dexterity")].values[0])
#convert the timestamp to a date
dexterity_history['timestamp'] = pd.to_datetime(dexterity_history['timestamp'], unit='s')
# save the data to a csv
dexterity_history.to_csv('..\\data\\Dexterity_tincture_market.csv', index=False)
dexterity_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2863,51,Lune Cr'est,False,2024-03-20 16:39:34,Adamantoise,73
1,True,2961,99,Shadow Erebus,False,2024-03-20 15:53:35,Siren,57
2,False,1500,3,Udi Khona,False,2024-03-20 11:39:14,Sargatanas,99
3,True,2645,99,Ryzuo Lesorenia,False,2024-03-20 11:23:50,Sargatanas,99
4,True,2645,99,Ryzuo Lesorenia,False,2024-03-20 11:23:49,Sargatanas,99
...,...,...,...,...,...,...,...,...
76130,True,14998,33,Shiro Greybourne,False,2023-05-23 14:57:09,Jenova,40
76131,True,14998,45,Shiro Greybourne,False,2023-05-23 14:57:08,Jenova,40
76132,True,15000,30,Cipher Mss,False,2023-05-23 13:06:54,Jenova,40
76133,True,15000,30,Cipher Mss,False,2023-05-23 13:06:52,Jenova,40


In [9]:
intelligence_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Intelligence")].values[0])
#convert the timestamp to a date
intelligence_history['timestamp'] = pd.to_datetime(intelligence_history['timestamp'], unit='s')
# save the data to a csv
intelligence_history.to_csv('..\\data\\Intelligence_tincture_market.csv', index=False)
intelligence_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2823,25,Coconut Melk,False,2024-03-20 16:22:20,Midgardsormr,65
1,False,2000,3,Deimos Drackin,False,2024-03-20 15:36:20,Gilgamesh,63
2,False,2997,15,Deimos Drackin,False,2024-03-20 15:36:04,Gilgamesh,63
3,True,2823,25,Faeron Evergleam,False,2024-03-20 15:13:57,Midgardsormr,65
4,True,2975,99,Polyphemian Devorare,False,2024-03-20 14:09:29,Faerie,54
...,...,...,...,...,...,...,...,...
65980,True,15000,10,Vox Veritatis,False,2023-05-23 17:13:01,Jenova,40
65981,True,30000,33,Camellia Yagami,False,2023-05-23 15:10:54,Siren,57
65982,True,30000,33,Camellia Yagami,False,2023-05-23 15:10:53,Siren,57
65983,True,30000,33,Camellia Yagami,False,2023-05-23 15:10:52,Siren,57


In [10]:
mind_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Mind")].values[0])
#convert the timestamp to a date
mind_history['timestamp'] = pd.to_datetime(mind_history['timestamp'], unit='s')
# save the data to a csv
mind_history.to_csv('..\\data\\Mind_tincture_market.csv', index=False)
mind_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,3089,99,Ren Ryder,False,2024-03-20 08:55:08,Faerie,54
1,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:38,Gilgamesh,63
2,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:37,Gilgamesh,63
3,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:36,Gilgamesh,63
4,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:35,Gilgamesh,63
...,...,...,...,...,...,...,...,...
74000,True,13000,27,Halo Nya,False,2023-05-23 13:55:49,Jenova,40
74001,True,40000,3,Atropia Shade,False,2023-05-23 13:28:50,Gilgamesh,63
74002,True,24999,9,Atropia Shade,False,2023-05-23 13:28:49,Gilgamesh,63
74003,True,19000,50,Atropia Shade,False,2023-05-23 13:28:47,Gilgamesh,63


In [11]:
tinctures

,ID,Icon,Name,Url,UrlType,_,_Score
0,39727,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0
